In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# read all data

games = pd.read_csv('games.csv')
games.drop_duplicates(inplace=True, subset=['GAME_ID'])
games_details = pd.read_csv('games_details.csv')
players = pd.read_csv('players.csv')
ranking = pd.read_csv('ranking.csv')
teams = pd.read_csv('teams.csv')

# 1. Data Preprocessing

In [3]:
# Choose season to analyze
# we will use 2015-16 throughout this report
this_season = 2015

### 1.1 Games

In [4]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24622 entries, 0 to 24676
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GAME_DATE_EST     24622 non-null  object 
 1   GAME_ID           24622 non-null  int64  
 2   GAME_STATUS_TEXT  24622 non-null  object 
 3   HOME_TEAM_ID      24622 non-null  int64  
 4   VISITOR_TEAM_ID   24622 non-null  int64  
 5   SEASON            24622 non-null  int64  
 6   TEAM_ID_home      24622 non-null  int64  
 7   PTS_home          24523 non-null  float64
 8   FG_PCT_home       24523 non-null  float64
 9   FT_PCT_home       24523 non-null  float64
 10  FG3_PCT_home      24523 non-null  float64
 11  AST_home          24523 non-null  float64
 12  REB_home          24523 non-null  float64
 13  TEAM_ID_away      24622 non-null  int64  
 14  PTS_away          24523 non-null  float64
 15  FG_PCT_away       24523 non-null  float64
 16  FT_PCT_away       24523 non-null  float6

In [5]:
# change game date string -> datetime
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'])

Get Regular Season Data

In [6]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime 

In [7]:
# to get 2015-16 season data, use season = 15
# but note that for 2019-20 season, the regular season was from 
# 10/22/19-03-11/20, 07/30/20-08/14/20 due to COVID
# so instead of season = 19, use the dates above and slice manually
# regular games for other seasons can be obtained with this function

def get_regular(season):
    
    #scrape regular season dates
    url = "https://en.wikipedia.org/wiki/20" + str(season) + "-" + str(season+1) + "_NBA_season"
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'lxml')
    season_dates = results_page.find('body').find('table', {'class':'infobox'}).find_all('tr')[3].find('td')
    
    pattern = r"(\w+)\s(\d+),\s(\d+)\W+(\w+)\s(\d+),\s(\d+)"
    me = re.search(pattern, str(season_dates))
    
    dates = (me.group(1), me.group(2), me.group(3), me.group(4), me.group(5), me.group(6))
    
    start_date = datetime.strptime(dates[0]+'/'+dates[1]+'/'+dates[2], '%B/%d/%Y')
    end_date = datetime.strptime(dates[3]+'/'+dates[4]+'/'+dates[5], '%B/%d/%Y')
    
    # obtain regular games for this season
    games_played_year = games[games['SEASON']==2000+season]
    games_regular_year = games_played_year[(start_date<=games_played_year['GAME_DATE_EST'])\
                                            & (games_played_year['GAME_DATE_EST']<=end_date)]
    
    return games_regular_year

# define a function that returns start and end dates (used later when computing elo)
def get_regular_dates(season):
    #scrape regular season dates
    url = "https://en.wikipedia.org/wiki/20" + str(season) + "-" + str(season+1) + "_NBA_season"
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'lxml')
    season_dates = results_page.find('body').find('table', {'class':'infobox'}).find_all('tr')[3].find('td')
    
    pattern = r"(\w+)\s(\d+),\s(\d+)\W+(\w+)\s(\d+),\s(\d+)"
    me = re.search(pattern, str(season_dates))
    
    dates = (me.group(1), me.group(2), me.group(3), me.group(4), me.group(5), me.group(6))
    
    start_date = datetime.strptime(dates[0]+'/'+dates[1]+'/'+dates[2], '%B/%d/%Y')
    end_date = datetime.strptime(dates[3]+'/'+dates[4]+'/'+dates[5], '%B/%d/%Y')
    
    return (start_date, end_date)

In [8]:
regular_games_2015 = get_regular(this_season-2000)

### 1.2 games_details

cleaning

In [9]:
# drop start position, comment
games_details.drop(columns = ['START_POSITION', 'COMMENT'], inplace = True)

# fill other nan with 0s - nan just means player did not play, filling with 0 makes sense
games_details.fillna(0, inplace = True)

# convert MIN object -> float(seconds)
games_details['MIN'] = games_details['MIN'].astype('str')

def format_min(string):
    if ':' in string:
        t = 60*int(string[:-3]) + int(string[-2:])
    else: 
        t = 60*int(string)
    return t

games_details['sec'] = games_details['MIN'].apply(lambda x: format_min(x))

get regular season data

In [10]:
games_details_master = games_details.join(games[['GAME_ID', 'SEASON', 'GAME_DATE_EST']].set_index('GAME_ID'), on = 'GAME_ID', how='left')

def get_regular_details(season):
    
    regular_games_season = get_regular(season)
    regular_games_details_season = games_details_master[games_details_master['GAME_ID'].isin(list(regular_games_season['GAME_ID']))]
    
    return regular_games_details_season

In [11]:
regular_games_details_2015 = get_regular_details(this_season-2000)

# 2. Feature Engineering

### 2.1 Aggregating Individual Statistics to the Game Level

Sum individual statistics to get game level statistics

In [12]:
regular_games_2015['winner_ID'] = np.where(regular_games_2015['HOME_TEAM_WINS']==1, regular_games_2015['HOME_TEAM_ID'], regular_games_2015['VISITOR_TEAM_ID'])

regular_games_stats_2015 = regular_games_details_2015.groupby(['GAME_ID', 'TEAM_ID'])[['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'REB', 'AST','OREB', 'DREB', 'STL', 'BLK', 'TO', 'PF', 'PTS']].sum().reset_index()
regular_games_stats_2015 = regular_games_stats_2015.join(regular_games_2015[['GAME_DATE_EST', 'GAME_ID', 'SEASON']].set_index('GAME_ID'), on = 'GAME_ID')
regular_games_stats_2015['order'] = regular_games_stats_2015.groupby('TEAM_ID')['GAME_DATE_EST'].rank(method='first')


We have two rows for each game. To combine them into a single row: <br>
Join home data for home team, away data for visitor team

In [13]:
# add column 'win' to distinguish home and away team
winner_table_2015 = regular_games_stats_2015[['GAME_ID', 'TEAM_ID', 'SEASON', 'order']].join(regular_games_2015[['GAME_ID', 'winner_ID']].set_index('GAME_ID'), how = 'left', on = 'GAME_ID')
winner_table_2015['win'] = np.where(winner_table_2015['TEAM_ID'] == winner_table_2015['winner_ID'], 1, 0)

regular_games_stats_2015 = regular_games_stats_2015.join(winner_table_2015[['GAME_ID', 'TEAM_ID', 'win']].set_index(['GAME_ID', 'TEAM_ID']), how = 'left', on = ['GAME_ID', 'TEAM_ID'])

# Join home, away data for corresponding team 
cols2 = ['GAME_ID', 'TEAM_ID', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'STL', 'BLK', 'TO', 'PF', 'order', 'win']
cols3 = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'STL', 'BLK', 'TO', 'PF', 'order', 'win']

cols3_dict = dict([(col, col+"_home") for col in cols3])
cols4_dict = dict([(col, col+"_away") for col in cols3])

regular_games_2015 = regular_games_2015.join(regular_games_stats_2015[cols2].set_index(['GAME_ID', 'TEAM_ID']), how = 'left', on = ['GAME_ID', 'HOME_TEAM_ID'])
regular_games_2015 = regular_games_2015.rename(columns = cols3_dict)
regular_games_2015 = regular_games_2015.join(regular_games_stats_2015[cols2].set_index(['GAME_ID', 'TEAM_ID']), how = 'left', on = ['GAME_ID', 'VISITOR_TEAM_ID'])
regular_games_2015 = regular_games_2015.rename(columns = cols4_dict)



Calulate efficiency for each game

In [14]:
regular_games_2015['Total_posessioins_home'] = regular_games_2015['FGA_home'] - regular_games_2015['OREB_home'] + regular_games_2015['TO_home'] + 0.4*regular_games_2015['FTA_home']
regular_games_2015['Total_posessioins_away'] = regular_games_2015['FGA_away'] - regular_games_2015['OREB_away'] + regular_games_2015['TO_away'] + 0.4*regular_games_2015['FTA_away']

regular_games_2015['Offensive_efficiency_home'] = regular_games_2015['PTS_home']/regular_games_2015['Total_posessioins_home']
regular_games_2015['Offensive_efficiency_away'] = regular_games_2015['PTS_away']/regular_games_2015['Total_posessioins_away']

regular_games_2015['Defensive_efficiency_home'] = regular_games_2015['PTS_away']/regular_games_2015['Total_posessioins_home']
regular_games_2015['Defensive_efficiency_away'] = regular_games_2015['PTS_home']/regular_games_2015['Total_posessioins_away']
# Higher value of Defensive efficiency = bad defense

Leave only necessary columns

In [15]:
# let's only get the columns that we need
regular_games_2015 = regular_games_2015[['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON',
        'HOME_TEAM_WINS',
       'Offensive_efficiency_home', 'Offensive_efficiency_away',
       'Defensive_efficiency_home', 'Defensive_efficiency_away']]

# sort by date
regular_games_2015.sort_values(by = 'GAME_DATE_EST', ascending=True, inplace=True)

Create a base table that will be used in aggregation of past data

In [16]:
base_2015 = regular_games_2015[['GAME_ID', 'GAME_DATE_EST', 'SEASON', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'HOME_TEAM_WINS']]

### 2.2 Aggregating Past Game Data

Now we compute features that will be used in EDA and modeling. <br>
To do that, we aggregate past games for each team this season. 

**1. Home Advantage** (Home over overall, away over overall)

In [17]:
games_dict = {}
teams_dict= {}
#every team id corresponds to [home total #, home win #, away total#, away won#]

# populate games_dict
for index, row in regular_games_2015.iterrows():
    if row["HOME_TEAM_ID"] not in teams_dict:
        teams_dict[row["HOME_TEAM_ID"]] = [0,0,0,0]
    if row["VISITOR_TEAM_ID"] not in teams_dict:
        teams_dict[row["VISITOR_TEAM_ID"]] = [0,0,0,0]
    if teams_dict[row["HOME_TEAM_ID"]][0] == 0:
        home= 0
        overall_hometeam = 0
    else:
        home = teams_dict[row['HOME_TEAM_ID']][1] / teams_dict[row['HOME_TEAM_ID']][0]
        overall_hometeam = (teams_dict[row['HOME_TEAM_ID']][1]+teams_dict[row['HOME_TEAM_ID']][3])/(teams_dict[row['HOME_TEAM_ID']][0] + teams_dict[row['HOME_TEAM_ID']][2])

    if teams_dict[row['VISITOR_TEAM_ID']][2] == 0:
        away = 0
        overall_awayteam = 0
    else:
        away = teams_dict[row['VISITOR_TEAM_ID']][3]/ teams_dict[row['VISITOR_TEAM_ID']][2]
        overall_awayteam = (teams_dict[row['VISITOR_TEAM_ID']][1]+teams_dict[row['VISITOR_TEAM_ID']][3])/(teams_dict[row['VISITOR_TEAM_ID']][0] + teams_dict[row['VISITOR_TEAM_ID']][2])
    games_dict[row['GAME_ID']] = (home, away, overall_hometeam, overall_awayteam)
    # games_dict                  home team's win rate at its home game
    #                                   away team's win rate at its away game
                                              # home team overall win rate
                                                                # away team overall win rate

    teams_dict[row['HOME_TEAM_ID']][0]+=1
    teams_dict[row['VISITOR_TEAM_ID']][2]+=1

    if row['HOME_TEAM_WINS'] == 1:
        teams_dict[row['HOME_TEAM_ID']][1] += 1
    elif row['HOME_TEAM_WINS']== 0:
        teams_dict[row['VISITOR_TEAM_ID']][3] += 1

# write data from games_dict to dataframe
base_2015["home_rate"] = [0]*len(base_2015)
base_2015["away_rate"] = [0]*len(base_2015)
base_2015["home_over_overall"] = [0]*len(base_2015)
base_2015["away_over_overall"] = [0]*len(base_2015)

for game_id, (home, away, overall_hometeam, overall_awayteam) in games_dict.items():
    base_2015.loc[base_2015['GAME_ID']==game_id,'home_rate'] = home
    base_2015.loc[base_2015['GAME_ID']==game_id,'away_rate'] = away
    base_2015.loc[base_2015['GAME_ID']==game_id,'home_over_overall'] = home/overall_hometeam if overall_hometeam != 0 else 0
    base_2015.loc[base_2015['GAME_ID']==game_id,'away_over_overall'] = away/overall_awayteam if overall_awayteam != 0 else 0


<ipython-input-17-275c22dd8cd1>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_2015["home_rate"] = [0]*len(base_2015)
<ipython-input-17-275c22dd8cd1>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_2015["away_rate"] = [0]*len(base_2015)
<ipython-input-17-275c22dd8cd1>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

**2. Momentum, Efficiencies** (We take the average of data from past 5 games to predict each game)

Prepare table for aggregation

In [18]:
stats_before_agg = regular_games_stats_2015[['GAME_ID', 'GAME_DATE_EST','SEASON','TEAM_ID', 'order', 'win']].merge(\
                                                regular_games_2015[['GAME_ID', 'HOME_TEAM_ID',\
                                                  'Offensive_efficiency_home', 'Defensive_efficiency_home']].rename(columns = \
                                                {'HOME_TEAM_ID':'TEAM_ID'}), on=['GAME_ID', 'TEAM_ID'], how='left').merge(\
                                regular_games_2015[['GAME_ID', 'VISITOR_TEAM_ID',\
                                'Offensive_efficiency_away', 'Defensive_efficiency_away']].rename(columns = \
                                {'VISITOR_TEAM_ID':'TEAM_ID'}), on=['GAME_ID', 'TEAM_ID'], how='left')

stats_before_agg['Offensive_efficiency'] = stats_before_agg['Offensive_efficiency_home'].fillna(0) + stats_before_agg['Offensive_efficiency_away'].fillna(0)
stats_before_agg['Defensive_efficiency'] = stats_before_agg['Defensive_efficiency_home'].fillna(0) + stats_before_agg['Defensive_efficiency_away'].fillna(0)

stats_before_agg.drop(columns = ['Offensive_efficiency_home', 'Defensive_efficiency_home', 'Offensive_efficiency_away', 'Defensive_efficiency_away'], inplace=True)

Aggregate past 5 games

In [19]:
team_ids = list(teams['TEAM_ID'])
cols_agg = ['win', 'Offensive_efficiency', 'Defensive_efficiency']
stats_agg = pd.DataFrame()

for team in team_ids: 
    df = stats_before_agg[stats_before_agg['TEAM_ID']==team]
    for col in cols_agg: 
        df[col+'_avg5'] = df[col].shift(1).rolling(5).mean()
    stats_agg = pd.concat([stats_agg, df], ignore_index=True)
    
stats_agg.drop(columns = cols_agg, inplace=True)

<ipython-input-19-aecd616f9dc1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col+'_avg5'] = df[col].shift(1).rolling(5).mean()


Join home, away data to corresponding team

In [20]:
cols5 = ['win_avg5', 'Offensive_efficiency_avg5', 'Defensive_efficiency_avg5']
cols6 = ['win_avg5', 'Offensive_efficiency_avg5', 'Defensive_efficiency_avg5', 'GAME_ID', 'TEAM_ID']

cols5_home_dict = dict([(col, col+"_home") for col in cols5])
cols5_away_dict = dict([(col, col+"_away") for col in cols5])

base_2015_new = base_2015.join(stats_agg[cols6].set_index(['GAME_ID', 'TEAM_ID']), how = 'left', on = ['GAME_ID', 'HOME_TEAM_ID'])
base_2015_new = base_2015_new.rename(columns = cols5_home_dict)
base_2015_new = base_2015_new.join(stats_agg[cols6].set_index(['GAME_ID', 'TEAM_ID']), how = 'left', on = ['GAME_ID', 'VISITOR_TEAM_ID'])
base_2015_new = base_2015_new.rename(columns = cols5_away_dict)

**3. Strength** (ELO)

Generator for datetime range

In [21]:
from datetime import timedelta

def datetime_range(start=None, end=None):
    span = end - start
    for i in range(span.days + 1):
        yield start + timedelta(days=i)

Helper function that calculates elo of next day

In [22]:
def get_elo_next(season, base_table, day, k):
    
    regular = get_regular(season-2000)
    
    games_today = regular[regular['GAME_DATE_EST']==day]
    elo_today = base_table[base_table['date']==day]
    
    elo_next_day = elo_today.copy()
    elo_next_day['date'] = day + timedelta(days=1)

    for index,row in games_today.iterrows():
        if row['HOME_TEAM_WINS'] ==1 :
            winner_id = row['HOME_TEAM_ID']
            loser_id = row['VISITOR_TEAM_ID']
        else: 
            winner_id = row['VISITOR_TEAM_ID']
            loser_id = row['HOME_TEAM_ID']

        winner_elo = float(elo_today[elo_today['TEAM_ID']==winner_id]['rating'])
        loser_elo = float(elo_today[elo_today['TEAM_ID']==loser_id]['rating'])

        winner_prob = (1.0 / (1.0 + 10**((loser_elo-winner_elo) / 400)))
        loser_prob = (1.0 / (1.0 + 10**((winner_elo-loser_elo) / 400)))

        winner_new_elo = winner_elo + k*(1-winner_prob)
        loser_new_elo = loser_elo + k*(0-loser_prob)

        elo_next_day.loc[elo_next_day['TEAM_ID']==winner_id, 'rating'] = winner_new_elo
        elo_next_day.loc[elo_next_day['TEAM_ID']==loser_id, 'rating'] = loser_new_elo
        
    return elo_next_day


Function that creates ELO of season

In [23]:
def get_elo_season(season, k):
    
    season_start, season_end = get_regular_dates(season-2000)

    teams_elo = teams[['TEAM_ID', 'ABBREVIATION']].copy()
    teams_elo['date'] = season_start
    teams_elo['rating'] = 1500
    
    drange = datetime_range(start=season_start, end=season_end)
    
    for day in drange:
        next_day_elo = get_elo_next(season,teams_elo,day,k)
        teams_elo = pd.concat([teams_elo, next_day_elo], ignore_index=True)
    
    return teams_elo

Compute ELO for 2015 season

In [24]:
teams_elo_2015 = get_elo_season(this_season,20)

Add ELO to base table

In [25]:
base_2015_final = base_2015_new.join(teams_elo_2015[['TEAM_ID', 'date', 'rating']].set_index(['TEAM_ID', 'date']), how = 'left', on = ['HOME_TEAM_ID', 'GAME_DATE_EST'])
base_2015_final = base_2015_final.rename(columns = {'rating':'elo_home'})
base_2015_final = base_2015_final.join(teams_elo_2015[['TEAM_ID', 'date', 'rating']].set_index(['TEAM_ID', 'date']), how = 'left', on = ['VISITOR_TEAM_ID', 'GAME_DATE_EST'])
base_2015_final = base_2015_final.rename(columns = {'rating':'elo_away'})

# 3. Convert tables into csv files for later use 

This is the table that will be used for EDA and modeling. There are 30 teams in the NBA, and 82 games for each team. 82*30/2 = 1230

In [26]:
base_2015_final.to_csv('reg_2015.csv', index=False)
base_2015_final

,GAME_ID,GAME_DATE_EST,SEASON,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_WINS,home_rate,away_rate,home_over_overall,away_over_overall,win_avg5_home,Offensive_efficiency_avg5_home,Defensive_efficiency_avg5_home,win_avg5_away,Offensive_efficiency_avg5_away,Defensive_efficiency_avg5_away,elo_home,elo_away
23158,21500003,2015-10-27,2015,1610612744,1610612740,1,0.000,0.000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000
23157,21500002,2015-10-27,2015,1610612741,1610612739,1,0.000,0.000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000
23156,21500001,2015-10-27,2015,1610612737,1610612765,0,0.000,0.000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000
23142,21500004,2015-10-28,2015,1610612753,1610612764,0,0.000,0.000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000
23143,21500005,2015-10-28,2015,1610612738,1610612755,1,0.000,0.000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21940,21501228,2016-04-13,2015,1610612747,1610612762,1,0.275,0.400,1.392188,0.810000,0.0,0.939859,1.131814,0.4,1.052562,1.014081,1297.919130,1509.965604
21941,21501229,2016-04-13,2015,1610612756,1610612746,1,0.325,0.600,1.196591,0.916981,0.4,1.013044,1.018009,1.0,1.106714,0.955229,1330.250343,1619.637738
21942,21501230,2016-04-13,2015,1610612757,1610612743,1,0.675,0.375,1.271512,0.920455,0.6,1.151729,1.141826,0.2,1.012882,1.113672,1562.671224,1439.808120
21937,21501225,2016-04-13,2015,1610612749,1610612754,0,0.575,0.450,1.411364,0.828409,0.2,0.999480,1.116903,0.8,1.135727,1.051745,1432.840377,1527.443124


This is the ELO for 2015-16 season

In [27]:
teams_elo_2015.to_csv('elo_2015.csv', index=False)
teams_elo_2015

,TEAM_ID,ABBREVIATION,date,rating
0,1610612737,ATL,2015-10-27,1500.000000
1,1610612738,BOS,2015-10-27,1500.000000
2,1610612740,NOP,2015-10-27,1500.000000
3,1610612741,CHI,2015-10-27,1500.000000
4,1610612742,DAL,2015-10-27,1500.000000
...,...,...,...,...
5125,1610612764,WAS,2016-04-14,1507.398643
5126,1610612765,DET,2016-04-14,1540.415062
5127,1610612766,CHA,2016-04-14,1575.674342
5128,1610612739,CLE,2016-04-14,1605.730038
